In [ ]:
import getpass
import os

In [ ]:
KEY = "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"
MISTRALAI_API_KEY ="XN1gUFU0bOnw0nDGERkN37MAOHwc4WdQ"

In [ ]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = KEY

if not os.getenv("MISTRALAI_API_KEY"):
    os.environ["MISTRALAI_API_KEY"] = MISTRALAI_API_KEY

In [11]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)
# from langchain_groq import ChatGroq

# llm = ChatGroq(
#     model="llama-3.1-8b-instant",
#     temperature=0,
#     max_tokens=128,
#     timeout=30,
# )

In [12]:
from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love apples."),
]

ai_msg = llm.invoke(messages)


msg = ai_msg.additional_kwargs.get('reasoning_content')

reasoning_msg = AIMessage(content=msg)
reasoning_msg.pretty_print()
# ai_msg = llm.invoke(messages)
# msg = ai_msg.additional_kwargs.get('reasoning_content')
# ai_msg.pretty_print()

================================== Ai Message ==================================

Okay, the user wants to translate "I love apples." into French. Let me start by breaking down the sentence. "I love" is the subject and verb. In French, "I" is "Je" and "love" is "aime". Then "apples" is the object. The French word for apples is "pommes". So putting it together, it should be "Je aime les pommes." Wait, but in French, after the verb "aimer", when using the pronoun "je", the verb needs to agree in number and person. Since "je" is first person singular, the correct conjugation is "aime". So "Je aime" is correct. But wait, sometimes in French, when the verb comes right after "je", you need to add a little "s" to make it "j'aime". Oh right! That's the elision. When "je" is followed by a vowel sound, you use "j'" instead of "je". Since "aime" starts with a vowel, it becomes "j'aime". So the correct translation is "J'aime les pommes." Let me double-check that. Yes, "j'aime" is the correct form h

In [ ]:
pip install -qU langchain-community 

In [ ]:
pip -q install faiss-cpu


In [ ]:
pip -q install langchain_mistralai

In [15]:
from langchain_mistralai import MistralAIEmbeddings

embeddings = MistralAIEmbeddings(
    model="mistral-embed",
    api_key= MISTRALAI_API_KEY,
)

In [16]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['5a9a981d-dc78-4cc0-ba44-05dfd280b6d5',
 '966bceab-5341-423a-8d9d-38dc3d8cb597',
 'c080fb09-7aa5-451c-bf25-26f5bc178e29',
 '5e8cc3e4-6fcb-4d22-93d9-416c12b7997e',
 'd8b67bd5-c6df-4064-b5c5-4822e0a3d9a0',
 '29a24d08-34fa-45e9-96da-9a8da8ad8c5d',
 '28fe959c-b0f3-4eeb-aa02-fb0a3cca5a7b',
 '4fbbe594-f681-4953-afde-affba47c26b0',
 'de90e7b9-485b-49e2-b8de-c325d661771b',
 'cf9647e6-0de2-405e-bf1a-5ddc738af2ef']

In [18]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [20]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.416781] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
